In [1]:
import json
import re
import nltk
from gensim import models
import pandas as pd
import keras
from keras import layers
from keras.models import Sequential
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np;np.random.seed(42)
from keras.layers import SimpleRNN, Dense

xnum = 0
ynum = 0
with open('Tr.json', encoding="iso-8859-1") as f:
    json_data = json.load(f, encoding="utf-8")
    regex = re.compile(r'[가-힣]+/NNG')
    noNNG = re.compile(r'[가-힣]+')
    reAns = re.compile(r'[0-9]+')
    x_train = []
    y_train = []
    for i in range(len(json_data)):
        TrainX = []
        TrainY = []
        for y in json_data[i]['answer']:
            ynum = ynum + 1
            y = reAns.findall(y)
            TrainY = TrainY + y
        #print(TrainY)
        y_train.append(TrainY)
        for j in json_data[i]['sentence']:
            sentence = j.encode('iso-8859-1').decode('euc-kr', 'ignore')
            matchWord = regex.findall(sentence)
            sentence = ''.join(matchWord)
            matchWord = noNNG.findall(sentence)
            TrainX = TrainX + matchWord
        x_train.append(TrainX)
        xnum = xnum + 1
        

Using TensorFlow backend.


In [2]:
x_data = []
for i in x_train:
    x_data.append(' '.join(i))
    
y_data = []
for a in y_train:
    y_data.append(a[2:3])
    
y_data_string = []
for c in y_data:
    y_data_string.append(''.join(c))

In [3]:
#TF-IDF 구하기
tfidf = TfidfVectorizer(max_features = 3000, lowercase=False)
x_voca = tfidf.fit_transform(x_data)
train_data = x_voca.toarray()

#피쳐는 3천개로 0~2999의 값을 가짐.
#행렬에서 가중치는 다를 것으로 예상함.

In [4]:

lb = LabelEncoder()
y = lb.fit_transform(y_data_string)
one_hot_lb = to_categorical(y)

In [11]:
model = Sequential();
model.add(layers.Dense(6,input_shape=(train_data.shape[1], ),activation = 'relu'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 6)                 18006     
Total params: 18,006
Trainable params: 18,006
Non-trainable params: 0
_________________________________________________________________


In [6]:
#5겹 교차 검증 하겠음
k_fold = 5

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.fit(train_data,one_hot_lb,epochs=100, batch_size=32,validation_split=0.2, verbose = 1)

Instructions for updating:
Use tf.cast instead.
Train on 2784 samples, validate on 696 samples
Epoch 1/100
2784/2784 [==============================] - 0s 98us/step - loss: 0.7781 - acc: 0.8333 - val_loss: 0.4854 - val_acc: 0.8333
Epoch 2/100
2784/2784 [==============================] - 0s 40us/step - loss: 0.4104 - acc: 0.8334 - val_loss: 0.3534 - val_acc: 0.8338
Epoch 3/100
2784/2784 [==============================] - 0s 40us/step - loss: 0.3074 - acc: 0.8408 - val_loss: 0.2896 - val_acc: 0.8527
Epoch 4/100
2784/2784 [==============================] - 0s 43us/step - loss: 0.2597 - acc: 0.8697 - val_loss: 0.2669 - val_acc: 0.8776
Epoch 5/100
2784/2784 [==============================] - 0s 45us/step - loss: 0.2311 - acc: 0.8921 - val_loss: 0.2493 - val_acc: 0.8954
Epoch 6/100
2784/2784 [==============================] - 0s 40us/step - loss: 0.2106 - acc: 0.9052 - val_loss: 0.2326 - val_acc: 0.9030
Epoch 7/100
2784/2784 [==============================] - 0s 45us/step - loss: 0.1948 - ac

2784/2784 [==============================] - 0s 47us/step - loss: 0.0777 - acc: 0.9486 - val_loss: 0.2621 - val_acc: 0.9119
Epoch 59/100
2784/2784 [==============================] - 0s 47us/step - loss: 0.0772 - acc: 0.9484 - val_loss: 0.2621 - val_acc: 0.9124
Epoch 60/100
2784/2784 [==============================] - 0s 46us/step - loss: 0.0765 - acc: 0.9487 - val_loss: 0.2610 - val_acc: 0.9124
Epoch 61/100
2784/2784 [==============================] - 0s 47us/step - loss: 0.0761 - acc: 0.9492 - val_loss: 0.2649 - val_acc: 0.9114
Epoch 62/100
2784/2784 [==============================] - 0s 51us/step - loss: 0.0755 - acc: 0.9486 - val_loss: 0.2677 - val_acc: 0.9109
Epoch 63/100
2784/2784 [==============================] - 0s 40us/step - loss: 0.0750 - acc: 0.9492 - val_loss: 0.2677 - val_acc: 0.9112
Epoch 64/100
2784/2784 [==============================] - 0s 45us/step - loss: 0.0744 - acc: 0.9488 - val_loss: 0.2680 - val_acc: 0.9114
Epoch 65/100
2784/2784 [==============================

In [7]:
#테스트 데이터 받아오기.
x_test = []
y_test = []
with open('Te.json', encoding="iso-8859-1") as f:
    json_data = json.load(f, encoding="utf-8")
    regex = re.compile(r'[가-힣]+/NNG')
    noNNG = re.compile(r'[가-힣]+')
    reAns = re.compile(r'[0-9]+')
    for i in range(len(json_data)):
        testX = []
        textY = []
        for y in json_data[i]['answer']:
            y = reAns.findall(y)
            textY = textY + y
        y_test.append(textY)
        for j in json_data[i]['sentence']:
            sentence = j.encode('iso-8859-1').decode('euc-kr', 'ignore')
            matchWord = regex.findall(sentence)
            sentence = ''.join(matchWord)
            matchWord = noNNG.findall(sentence)
            testX = testX + matchWord
        x_test.append(testX)
        

In [8]:
x_data_test = []
for i in x_test:
    x_data_test.append(' '.join(i))
    
y_data_test = []
for a in y_test:
    y_data_test.append(a[2:3])
print
    
y_data_test_string = []
for c in y_data_test:
    y_data_test_string.append(''.join(c))
print(len(y_data_test_string))
print(len(x_test))
    
y_lb = lb.fit_transform(y_data_test_string)ㅇ
one_hot_lb_test = to_categorical(y_lb)
print(one_hot_lb_test.shape)

1497
1497
(1497, 6)


In [9]:
x_data_test = tfidf.transform(x_data_test)
x_data_test = x_data_test.toarray()
y_pred = model.predict(x_data_test,batch_size=32)

In [10]:
print(model.evaluate(x_data_test, one_hot_lb_test, batch_size=32)[1])

1497/1497 [==============================] - 0s 20us/step
0.9028056047722428
